In [ ]:
%cd ..

In [ ]:
import sys

sys.path.extend(('src', 'lib'))

In [ ]:
from pathlib import Path
from importlib import reload
import csv
import math
import itertools

import numpy as np
import torch
from tqdm.auto import tqdm
import voyager
import cv2
from matplotlib import pyplot as plt
from pytorch_metric_learning import losses

import voc, fcom
import data, utils, predictors, embedding, embedding_img

In [ ]:
data_dir = Path('data')
models_dir = Path('models')
embeddings_dir = Path('voc_embeddings')

In [ ]:
train_ds = data.Dataset(
    data.BinaryDataset.load('data/train.npz'),
    data.read_words(data_dir / 'train.ref'),
)
val_ds = data.Dataset(
    data.BinaryDataset.load('data/valid.npz'),
    data.read_words(data_dir / 'valid.ref'),
)
val_500_ds = data.Dataset(
    data.BinaryDataset.load(data_dir / 'valid_500.npz'),
    data.read_words(data_dir / 'valid_500.ref'),
)

vocabulary = utils.Vocabulary.load(data_dir / 'vocabulary.csv')
keyboard_grids = utils.load_grids(data_dir / 'grids.json')

In [ ]:
preprocessor = embedding.SeqPreprocessor(keyboard_grids, vocabulary)
train_pair_ds = embedding.TargetPairDataset(train_ds, vocabulary, preprocessor, 500)
val_pair_ds = embedding.TargetPairDataset(val_ds, vocabulary, preprocessor, 500)
val_500_pair_ds = embedding.TargetPairDataset(val_500_ds, vocabulary, preprocessor, 500)

In [ ]:
reload(data)
reload(utils)
reload(embedding)

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
model_name = 'emb_tf_v0_best'
model_path = models_dir / f'{model_name}.pt'
model = embedding.TransformerEmbedder(emb_dim=64, n_heads=4, n_layers=4).cuda()
opt = torch.optim.Adam(model.parameters(), lr=8e-5)
scheduler = torch.optim.lr_scheduler.StepLR(opt, 10, .5, verbose=True)

In [ ]:
embedding.train(
    model, opt, scheduler,
    train_pair_ds, val_pair_ds, 
    100, 100, 
    losses.ContrastiveLoss(), embedding.center_dist_penalty, .0,
    model_path
)

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
model.load_state_dict(torch.load(model_path))
model = model.eval()

In [ ]:
all_words = data.read_words(data_dir / 'voc.txt')
max(map(len, all_words))

In [ ]:
grid_word_embeddings = embedding.calculate_word_embeddings(model, all_words, keyboard_grids, preprocessor, 5000)

In [ ]:
np.savez(embeddings_dir / f'{model_name}.npz', **grid_word_embeddings)

In [ ]:
model = model.cpu()

In [ ]:
model = embedding.LSTMEmbedder()
model.load_state_dict(torch.load('models/emb_lstm_v0.pt'))
grid_word_embeddings = np.load('voc_embeddings/emb_lstm_v0.npz')

In [ ]:
reload(data)
reload(utils)
reload(embedding)

In [ ]:
candidate_gen = embedding.EmbeddingCandidateGenerator(model, grid_word_embeddings, preprocessor, vocabulary, 500, 64, 8)

In [ ]:
candidate_gen.n_candidates = 1000

In [ ]:
for i in itertools.islice(val_500_ds, 30):
    cands = candidate_gen(i.trace)
    cand_words = [c.word for c in cands]
    print(i.word in cand_words, i.word)#, ','.join(cand_words))

In [ ]:
flags = []
for i in itertools.islice(val_500_ds, 100):
    cands = candidate_gen(i.trace)
    cand_words = [c.word for c in cands]
    flags.append(i.word in cand_words)
    # print(i.word in cand_words, i.word, ','.join(cand_words))
np.mean(flags)

In [ ]:
def default_emedding(word):
    return grid_word_embeddings['default'][vocabulary.words.index(word)]

In [ ]:
np.linalg.norm(default_emedding('побег') - default_emedding('пробег'))